In [1]:

# update or install the necessary libraries
!pip install --upgrade openai
!pip install --upgrade langchain
!pip install --upgrade python-dotenv


Defaulting to user installation because normal site-packages is not writeable


In [42]:
from dotenv import load_dotenv
import openai
# Load environment variables from .env file

import os
load_dotenv()
OPENAI_API_KEY = os.getenv("openai_key")

In [46]:
from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": "Write a one-sentence bedtime story about a unicorn."
        }
    ],
    temperature=0.1,
)

print(completion.choices[0].message.content)

As the moonlight danced on the shimmering lake, a gentle unicorn named Luna spread her iridescent wings and soared through the starry sky, spreading dreams of magic and wonder to all the sleeping children below.


In [49]:
def set_open_params(
    model="gpt-4o-mini",
    temperature=0.7,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
):
    """ set openai parameters"""

    openai_params = {}    

    openai_params['model'] = model
    openai_params['temperature'] = temperature
    openai_params['max_tokens'] = max_tokens
    openai_params['top_p'] = top_p
    openai_params['frequency_penalty'] = frequency_penalty
    openai_params['presence_penalty'] = presence_penalty
    return openai_params

def get_completion(params, messages):
    """ GET completion from openai api"""

    response = client.chat.completions.create(
        model = params['model'],
        messages = messages,
        temperature = params['temperature'],
        max_tokens = params['max_tokens'],
        top_p = params['top_p'],
        frequency_penalty = params['frequency_penalty'],
        presence_penalty = params['presence_penalty'],
    )
    return response



In [50]:
# basic example
params = set_open_params()

prompt = "The sky is"

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)

In [51]:
response.choices[0].message.content

'The sky is a vast expanse above us, often filled with clouds, sunlight, or stars depending on the time of day. It can reflect a range of colors during sunrise and sunset, from soft pastels to vibrant hues. The sky plays a crucial role in weather patterns and is home to various phenomena such as rainbows, lightning, and the movement of celestial bodies. What specific aspect of the sky are you interested in?'

## Try with different temperature to compare results:

In [37]:
params = set_open_params(temperature=0)
response = get_completion(params, messages)
response.choices[0].message.content

'The sky is a vast expanse above us, often filled with clouds, stars, and the sun or moon. It can change colors throughout the day, from the bright blue of midday to the warm hues of sunrise and sunset. The sky also plays a crucial role in weather patterns and is home to various atmospheric phenomena. What specific aspect of the sky are you interested in?'

## 1.1 Text summarization


In [53]:
params = set_open_params(temperature=0.7)
prompt = """Antibiotics are a type of medication used to treat bacterial infections. They work by either killing the bacteria or preventing them from reproducing, allowing the body's immune system to fight off the infection. Antibiotics are usually taken orally in the form of pills, capsules, or liquid solutions, or sometimes administered intravenously. They are not effective against viral infections, and using them inappropriately can lead to antibiotic resistance. 

Explain the above in one sentence"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
response.choices[0].message.content

'Antibiotics are medications that treat bacterial infections by killing bacteria or inhibiting their reproduction, but they are ineffective against viruses and can contribute to antibiotic resistance if misused.'

### Exercise: Instruct the model to explain the paragraph in one sentence like "I am 5". Do you see any differences?

## 1.2 Question Answering

In [54]:
prompt = """Answer the question based on the context below. Keep the answer short and concise. Respond "Unsure about answer" if not sure about the answer.

Context: Teplizumab traces its roots to a New Jersey drug company called Ortho Pharmaceutical. There, scientists generated an early version of the antibody, dubbed OKT3. Originally sourced from mice, the molecule was able to bind to the surface of T cells and limit their cell-killing potential. In 1986, it was approved to help prevent organ rejection after kidney transplants, making it the first therapeutic antibody allowed for human use.

Question: What was OKT3 originally sourced from?

Answer:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
response.choices[0].message.content

'Mice'

## 1.3 Text classification

In [55]:
prompt = """Classify the text into neutral, negative or positive.

Text: I think the food was okay.

Sentiment:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
response.choices[0].message.content

Exercise: Modify the prompt to instruct the model to provide an explanation to the answer selected.

In [56]:
## 1.4 Reasoning

In [58]:
prompt = """The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 

Solve by breaking the problem into steps. First, identify the odd numbers, add them, and indicate whether the result is odd or even."""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
print(response.choices[0].message.content)

Let's break the problem down into steps:

### Step 1: Identify the odd numbers in the group

The given numbers are: 15, 32, 5, 13, 82, 7, 1.

The odd numbers from this group are:
- 15
- 5
- 13
- 7
- 1

### Step 2: Add the identified odd numbers

Now, let's add the odd numbers together:

\[
15 + 5 + 13 + 7 + 1
\]

Calculating step by step:
- \(15 + 5 = 20\)
- \(20 + 13 = 33\)
- \(33 + 7 = 40\)
- \(40 + 1 = 41\)

So, the sum of the odd numbers is \(41\).

### Step 3: Determine whether the result is odd or even

The result, \(41\), is an odd number.

### Conclusion

The odd numbers in the group add up to \(41\), which is odd, not even. Therefore, the statement "The odd numbers in this group add up to an even number" is incorrect.


# 2. Advanced Prompting Techniques

## 2.1 Few-shots prompt

In [59]:
prompt = """The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: The answer is False.

The odd numbers in this group add up to an even number: 17,  10, 19, 4, 8, 12, 24.
A: The answer is True.

The odd numbers in this group add up to an even number: 16,  11, 14, 4, 8, 13, 24.
A: The answer is True.

The odd numbers in this group add up to an even number: 17,  9, 10, 12, 13, 4, 2.
A: The answer is False.

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 
A:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
print(response.choices[0].message.content)

To determine if the sum of the odd numbers in the group adds up to an even number, we first identify the odd numbers from the group provided: 

The odd numbers in the group 15, 32, 5, 13, 82, 7, 1 are:
- 15
- 5
- 13
- 7
- 1

Now, we calculate the sum of these odd numbers:

15 + 5 = 20  
20 + 13 = 33  
33 + 7 = 40  
40 + 1 = 41  

The sum of the odd numbers is 41, which is an odd number.

Therefore, the answer is **False**.


## 2.2 Chain-of-Thought (CoT) Prompting

In [60]:
prompt = """The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: Adding all the odd numbers (9, 15, 1) gives 25. The answer is False.

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 
A:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
print(response.choices[0].message.content)

Let's identify the odd numbers in the group: 15, 5, 13, 7, 1.

Now, we add them up:

15 + 5 + 13 + 7 + 1 = 41.

Since 41 is an odd number, the statement is False. The odd numbers in this group do not add up to an even number.


## 2.3 Zero-shot CoT

In [62]:
prompt = """I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the repairman. I then went and bought 5 more apples and ate 1. How many apples did I remain with?

Let's think step by step."""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
print(response.choices[0].message.content)

Let's break it down step by step:

1. You started with 10 apples.
2. You gave 2 apples to the neighbor: 
   - 10 - 2 = 8 apples left.
3. You gave 2 apples to the repairman: 
   - 8 - 2 = 6 apples left.
4. You bought 5 more apples: 
   - 6 + 5 = 11 apples left.
5. You ate 1 apple: 
   - 11 - 1 = 10 apples left.

So, you remain with **10 apples**.


## 2.4 Self consistency

In [67]:
from collections import Counter


cot_prompt = """What is 17 times 4? Let's think step by step."""

# Generate multiple CoT reasoning paths
answers = []
for _ in range(5):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": cot_prompt + " Answer:"}],
        max_tokens=200
    )
    print(response.choices[0].message.content)
    answers.append(response.choices[0].message.content.split()[-1])  # Take final answer word

 
# Vote for the most common answer
print(Counter(answers).most_common(1))


To calculate \(17 \times 4\) step by step, you can break it down like this:

1. Start with the number 17.
2. Multiply it by 4. You can do this by adding 17 four times:
   - \(17 + 17 = 34\)
   - \(34 + 17 = 51\)
   - \(51 + 17 = 68\)

So, \(17 \times 4 = 68\).

Therefore, the answer is **68**.
To calculate \( 17 \times 4 \) step by step, you can break it down as follows:

1. Start with the number 17.
2. Multiply 17 by 4.

You can think of it as adding 17 four times:

\[
17 + 17 + 17 + 17
\]

Calculating that step-by-step:

- First, \( 17 + 17 = 34 \)
- Next, add another 17: \( 34 + 17 = 51 \)
- Finally, add the last 17: \( 51 + 17 = 68 \)

So, \( 17 \times 4 = 68 \).

Therefore, the answer is \( 68 \).
To calculate \( 17 \times 4 \), we can break it down step by step:

1. **Understanding the multiplication**: Multiplication is essentially adding a number to itself a certain number of times. In this case, we are adding 17 four times.

2. **Calculating step by step**:
   - First, calcula

In [68]:
Counter(answers)

Counter({'**68**.': 3, '\\).': 1, '68': 1})

## 2.5 Prompt chaining

In [70]:
# Prompt Chaining: Decompose a task into sub-tasks
doc = """
Artificial Intelligence (AI): A New Era of Innovation

Artificial Intelligence, commonly referred to as AI, represents one of the most transformative technologies of the 21st century. At its core, AI enables machines to mimic human cognitive functions such as learning, reasoning, problem-solving, and decision-making. From self-driving cars to intelligent virtual assistants, AI systems are increasingly embedded into our daily lives, reshaping industries like healthcare, finance, education, and entertainment.

Recent advancements in deep learning, natural language processing, and computer vision have accelerated AI's capabilities, allowing machines not only to perform complex tasks but also to generate creative content, understand emotions, and interact more naturally with humans. As AI continues to evolve, it holds immense potential to solve global challenges — from predicting climate patterns to personalizing medicine.

However, with this great power comes the responsibility to address ethical considerations, such as fairness, transparency, and accountability. Ensuring that AI systems are developed and deployed responsibly will be key to unlocking their full benefits for society.


"""

# Step 1: Extract key facts
extract = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": f"Extract three key facts from the text:\n{doc}"}],
    max_tokens=200
)
facts = extract.choices[0].message.content

# Step 2: Summarize the extracted facts
summary = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": f"Summarize these facts in one sentence:\n{facts}"}],
    max_tokens=300
)

print(summary.choices[0].message.content.strip())


AI mimics human cognitive functions and is increasingly integrated into various industries, with recent advancements enhancing its capabilities, while ethical considerations of fairness, transparency, and accountability are essential for its responsible development and societal benefits.
